# Análisis de resultados de los experimentos

#### descarga de todas las runs del pryecto

In [1]:
import pandas as pd
import wandb

api = wandb.Api()
entity, project = "inaki", "sacyr"
runs = api.runs(entity + "/" + project)

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values
    #  for metrics such as accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)
# unwrap the summary and config dictionaries
runs_df = runs_df.join(pd.json_normalize(runs_df['summary'])).drop(columns=['summary'])
runs_df = runs_df.join(pd.json_normalize(runs_df['config'])).drop(columns=['config'])

runs_df.to_csv("project.csv")

display(runs_df.head())

,name,Mean_Test_loss,Mean_Test_mae,Mean_Val_loss,Mean_Val_mae,Train_loss,Val_loss,Val_mae,_runtime,_step,...,model.num_layers,model.d_embedding,model.hidden_size,model.n_frequencies,model.frequency_init_scale,model.cnn_filters,model.kernel_size,model.nhead,model.ff_dim,model.embed_dim_per_head
0,denim-sweep-1,0.242657,0.366385,0.151487,0.295540,0.1887,0.153333,0.186739,101.477598,156.0,...,4,253.0,392.0,47.0,0.20,NaN,NaN,NaN,NaN,NaN
1,lucky-sweep-1,0.092526,0.226757,0.037514,0.136340,0.0414,0.036562,0.098513,118.500976,182.0,...,3,101.0,990.0,27.0,1.00,NaN,NaN,NaN,NaN,NaN
2,expert-sweep-1,0.115034,0.262239,0.057491,0.174612,0.0567,0.067077,0.160589,116.241957,182.0,...,3,217.0,703.0,16.0,5.00,NaN,NaN,NaN,NaN,NaN
3,jumping-sweep-1,0.046913,0.184895,0.032632,0.134321,0.0450,0.030435,0.108974,121.731029,171.0,...,4,58.0,897.0,35.0,0.05,NaN,NaN,NaN,NaN,NaN
4,jolly-sweep-1,0.042193,0.199024,0.030048,0.132889,0.0258,0.028902,0.126539,108.992188,131.0,...,4,181.0,474.0,44.0,1.00,NaN,NaN,NaN,NaN,NaN


In [2]:
runs_df['model.name'].unique()  # List unique model names

array(['periodical_mlp', 'lstm', 'cnn_lstm', 'transformer_base'],
      dtype=object)

In [3]:
import pandas as pd

df = (
  runs_df[runs_df['model.name'] == "periodical_mlp"]
    .sort_values('Mean_Val_mae')
    .groupby('dataset.name')
    .head(10)
    .groupby('dataset.name')['Mean_Test_mae']
    .mean()
    .reset_index(name='Mean_Test_mae_top10')
    .sort_values('Mean_Test_mae_top10', ascending=True)
    .reset_index(drop=True)
)

print(df)



                 dataset.name  Mean_Test_mae_top10
0            108+43-24h-1h-2c             0.084818
1            108+43-12h-1h-2c             0.090034
2            108+43-48h-1h-2c             0.091403
3    108+43+3493-24h-1h-1L-2c             0.093942
4   108+43+3493-24h-1h-mes-2c             0.095019
..                        ...                  ...
95          baseline-1h-1h-2c             0.326467
96            108+43-1h-1h-1c             0.337469
97            108+43-6h-1h-1c             0.401256
98             baseline-1h-6h             0.485009
99           108+43-12h-1h-1c             0.833682

[100 rows x 2 columns]


## Resultados experimentos baseline

In [4]:
# --- partimos del df que ya tienes ---
# df.columns -> ['dataset.name', 'Mean_Test_mae_top10']

# filtramos por datasets.name que empiezan por baseline
df_base = df[df['dataset.name'].str.startswith('baseline')].copy()

# 1. Separar sufijo "-1c" / "-2c" (o «base» si no hay sufijo)
df_base['variant']   = df_base['dataset.name'].str.extract(r'-(\d+c)$', expand=False).fillna('base')
df_base['base_name'] = df_base['dataset.name'].str.replace(r'-(\d+c)$', '', regex=True)

# 2. Pivotar para que cada variante quede en una columna
pivot = (
    df_base.pivot(index='base_name',            # una fila por dataset «base»
             columns='variant',            # columnas: base, 1c, 2c
             values='Mean_Test_mae_top10') # los valores numéricos
      .reset_index()
)

# 3. Añadir las columnas de texto que pides (nombres completos de cada variante)
pivot['dataset_base'] = pivot['base_name']
pivot['dataset_1c']   = pivot['base_name'] + '-1c'
pivot['dataset_2c']   = pivot['base_name'] + '-2c'

# 4. Reordenar / renombrar para la tabla final
resultado = (
    pivot
      .rename(columns={'base':'Mean_base', '1c':'Mean_1c', '2c':'Mean_2c'})
      [['dataset_base', 'Mean_base',
        'dataset_1c',  'Mean_1c',
        'dataset_2c',  'Mean_2c']]
)

display(resultado)


variant,dataset_base,Mean_base,dataset_1c,Mean_1c,dataset_2c,Mean_2c
0,baseline-12h-1h,0.167910,baseline-12h-1h-1c,0.136519,baseline-12h-1h-2c,0.135637
1,baseline-12h-3h,0.185581,baseline-12h-3h-1c,NaN,baseline-12h-3h-2c,NaN
2,baseline-12h-6h,0.191281,baseline-12h-6h-1c,NaN,baseline-12h-6h-2c,NaN
3,baseline-1h-1h,0.310572,baseline-1h-1h-1c,0.315209,baseline-1h-1h-2c,0.326467
4,baseline-1h-3h,0.313567,baseline-1h-3h-1c,NaN,baseline-1h-3h-2c,NaN
5,baseline-1h-6h,0.485009,baseline-1h-6h-1c,NaN,baseline-1h-6h-2c,NaN
6,baseline-24h-1h,0.141597,baseline-24h-1h-1c,0.141353,baseline-24h-1h-2c,0.122137
7,baseline-24h-24h,NaN,baseline-24h-24h-1c,0.137980,baseline-24h-24h-2c,0.141382
8,baseline-24h-24h-1L,NaN,baseline-24h-24h-1L-1c,0.120698,baseline-24h-24h-1L-2c,0.119480
9,baseline-24h-24h-2L,NaN,baseline-24h-24h-2L-1c,0.114468,baseline-24h-24h-2L-2c,0.116763


## Resultados añadiendo espiras:   108+43  &  108+3493

In [5]:
import pandas as pd

# --- 1) filtrar las runs que empiezan por 108+ ---
df108 = df[
    (df['dataset.name'].str.startswith('108+43-') |
    df['dataset.name'].str.startswith('108+3493')) &
    ~df['dataset.name'].str.contains(r'1L|2L|3L|mes') # excluimos las que contienen 1L, 2L, 3L o mes
].copy()

# --- 2) mismo tratamiento que antes ---
rows = []
for base, g in df108.assign(
        variant   = lambda x: x['dataset.name'].str.extract(r'-(\d+c)$')[0]
                               .fillna('base'),
        base_name = lambda x: x['dataset.name'].str.replace(r'-(\d+c)$', '', regex=True)
).groupby('base_name'):

    fila = {
        'dataset_1c'  : f'{base}-1c',         # 108+…-1c
        'Mean_1c'     : g.loc[g['variant']=='1c',  'Mean_Test_mae_top10'].squeeze()
                        if '1c'  in g['variant'].values else pd.NA,

        'dataset_2c'  : f'{base}-2c',         # 108+…-2c
        'Mean_2c'     : g.loc[g['variant']=='2c',  'Mean_Test_mae_top10'].squeeze()
                        if '2c'  in g['variant'].values else pd.NA,
    }
    rows.append(fila)

resultado_108 = pd.DataFrame(rows)

# Añado columna mean de la columna Mean_1c y Mean_2c
resultado_108['Mean_error'] = (resultado_108['Mean_1c'] + resultado_108['Mean_2c']) / 2
# «resultado_108» ya es la tabla con 6 columnas solicitadas
display(resultado_108)


,dataset_1c,Mean_1c,dataset_2c,Mean_2c,Mean_error
0,108+3493-12h-1h-1c,0.130693,108+3493-12h-1h-2c,0.134027,0.132360
1,108+3493-1h-1h-1c,0.226814,108+3493-1h-1h-2c,0.237297,0.232056
2,108+3493-24h-1h-1c,0.134268,108+3493-24h-1h-2c,0.128038,0.131153
3,108+3493-3h-1h-1c,0.158891,108+3493-3h-1h-2c,0.188752,0.173821
4,108+3493-48h-1h-1c,0.144055,108+3493-48h-1h-2c,0.147163,0.145609
5,108+3493-6h-1h-1c,0.137431,108+3493-6h-1h-2c,0.154086,0.145759
6,108+3493-72h-1h-1c,0.175029,108+3493-72h-1h-2c,0.154353,0.164691
7,108+43-12h-1h-1c,0.833682,108+43-12h-1h-2c,0.090034,0.461858
8,108+43-1h-1h-1c,0.337469,108+43-1h-1h-2c,0.134839,0.236154
9,108+43-24h-1h-1c,0.130717,108+43-24h-1h-2c,0.084818,0.107768


# Pruebas con las 3 espiras

In [6]:
import pandas as pd

# --- 1) filtrar las runs que empiezan por 108+ ---
df108 = df[
    df['dataset.name'].str.startswith('108+43+3493') &
    ~df['dataset.name'].str.contains(r'1L|2L|3L|mes') # excluimos las que contienen 1L, 2L, 3L o mes
].copy()

# --- 2) mismo tratamiento que antes ---
rows = []
for base, g in df108.assign(
        variant   = lambda x: x['dataset.name'].str.extract(r'-(\d+c)$')[0]
                               .fillna('base'),
        base_name = lambda x: x['dataset.name'].str.replace(r'-(\d+c)$', '', regex=True)
).groupby('base_name'):

    fila = {
        'dataset_1c'  : f'{base}-1c',         # 108+…-1c
        'Mean_1c'     : g.loc[g['variant']=='1c',  'Mean_Test_mae_top10'].squeeze()
                        if '1c'  in g['variant'].values else pd.NA,

        'dataset_2c'  : f'{base}-2c',         # 108+…-2c
        'Mean_2c'     : g.loc[g['variant']=='2c',  'Mean_Test_mae_top10'].squeeze()
                        if '2c'  in g['variant'].values else pd.NA,
    }
    rows.append(fila)

resultado_108 = pd.DataFrame(rows)
# Añado columna mean de la columna Mean_1c y Mean_2c
resultado_108['Mean_error'] = (resultado_108['Mean_1c'] + resultado_108['Mean_2c']) / 2

# «resultado_108» ya es la tabla con 6 columnas solicitadas
display(resultado_108)

,dataset_1c,Mean_1c,dataset_2c,Mean_2c,Mean_error
0,108+43+3493-24h-1h-1c,0.149326,108+43+3493-24h-1h-2c,0.098791,0.124059
1,108+43+3493-48h-1h-1c,0.195965,108+43+3493-48h-1h-2c,0.096709,0.146337
2,108+43+3493-72h-1h-1c,0.270064,108+43+3493-72h-1h-2c,<NA>,<NA>


### Pruebas con features adicionales es_laborable (1L, 2L, 3L) y mes

In [7]:
df108 = df[
    df['dataset.name'].str.startswith('108+43+3493') &
    df['dataset.name'].str.contains(r'1L|2L|3L|mes') # excluimos las que contienen 1L, 2L, 3L o mes
].copy()

# --- 2) mismo tratamiento que antes ---
rows = []
for base, g in df108.assign(
        variant   = lambda x: x['dataset.name'].str.extract(r'-(\d+c)$')[0]
                               .fillna('base'),
        base_name = lambda x: x['dataset.name'].str.replace(r'-(\d+c)$', '', regex=True)
).groupby('base_name'):

    fila = {
        'dataset_1c'  : f'{base}-1c',         # 108+…-1c
        'Mean_1c'     : g.loc[g['variant']=='1c',  'Mean_Test_mae_top10'].squeeze()
                        if '1c'  in g['variant'].values else pd.NA,

        'dataset_2c'  : f'{base}-2c',         # 108+…-2c
        'Mean_2c'     : g.loc[g['variant']=='2c',  'Mean_Test_mae_top10'].squeeze()
                        if '2c'  in g['variant'].values else pd.NA,
    }
    rows.append(fila)

resultado_108 = pd.DataFrame(rows)
# Añado columna mean de la columna Mean_1c y Mean_2c
resultado_108['Mean_error'] = (resultado_108['Mean_1c'] + resultado_108['Mean_2c']) / 2

# «resultado_108» ya es la tabla con 6 columnas solicitadas
display(resultado_108)

print("Se observa que ninguna de las versiones con features extra mejora el error medio de las predicciones, por lo que se descartan para el modelo final.")

,dataset_1c,Mean_1c,dataset_2c,Mean_2c,Mean_error
0,108+43+3493-24h-1h-1L-1c,0.156699,108+43+3493-24h-1h-1L-2c,0.093942,0.125320
1,108+43+3493-24h-1h-2L-1c,0.157600,108+43+3493-24h-1h-2L-2c,0.101129,0.129365
2,108+43+3493-24h-1h-3L-1c,0.160917,108+43+3493-24h-1h-3L-2c,0.102266,0.131592
3,108+43+3493-24h-1h-mes-1c,0.162220,108+43+3493-24h-1h-mes-2c,0.095019,0.128620
4,108+43+3493-48h-1h-1L-1c,0.209648,108+43+3493-48h-1h-1L-2c,0.103185,0.156417
5,108+43+3493-48h-1h-2L-1c,0.220012,108+43+3493-48h-1h-2L-2c,0.116573,0.168292
6,108+43+3493-48h-1h-3L-1c,0.205675,108+43+3493-48h-1h-3L-2c,0.101027,0.153351
7,108+43+3493-48h-1h-mes-1c,0.203721,108+43+3493-48h-1h-mes-2c,0.105209,0.154465
8,108+43+3493-72h-1h-1L-1c,0.317361,108+43+3493-72h-1h-1L-2c,0.123992,0.220676
9,108+43+3493-72h-1h-2L-1c,0.296551,108+43+3493-72h-1h-2L-2c,0.105412,0.200981


Se observa que ninguna de las versiones con features extra mejora el error medio de las predicciones, por lo que se descartan para el modelo final.


# Comparacion entre modelos
### Hemos seleccionado 108 + 43 con 24h de contexto y predicción a 1h
### A continuacion se explora la comparacion sobre ese dataset de varios modelos mas especificos para series temporales 
### Comparamos con el modelo base "periodical_mlp" cuya media de Test_Mae es = 0.0927

In [8]:
df = (
  runs_df[runs_df['model.name'] != "periodical_mlp" ]
    .sort_values('Mean_Val_mae')
    .groupby('model.name')
    .head(10)
    .groupby('model.name')['Mean_Test_mae']
    .mean()
    .reset_index(name='Mean_Test_mae_top10')
    .sort_values('Mean_Test_mae_top10', ascending=True)
    .reset_index(drop=True)
)

df

,model.name,Mean_Test_mae_top10
0,cnn_lstm,0.078717
1,transformer_base,0.081163
2,lstm,0.083680


In [9]:
df = (
  runs_df[
    (runs_df['model.name'] != "periodical_mlp") &
    (runs_df['dataset.name'].str.startswith("108+43-24h-1h"))
  ]
    .sort_values('Mean_Val_mae')
    .groupby(['model.name', 'dataset.name'])
    .head(10)
    .groupby(['model.name', 'dataset.name'])['Mean_Test_mae']
    .mean()
    .reset_index(name='Mean_Test_mae_top10')
    .sort_values('Mean_Test_mae_top10', ascending=True)
    .reset_index(drop=True)
)
display(df)

df = (
  runs_df[
    (runs_df['model.name'] == "transformer_base")
  ]
    .sort_values('Mean_Val_mae')
    .groupby(['model.name', 'dataset.name'])
    .head(10)
    .groupby(['model.name', 'dataset.name'])['Mean_Test_mae']
    .mean()
    .reset_index(name='Mean_Test_mae_top10')
    .sort_values('Mean_Test_mae_top10', ascending=True)
    .reset_index(drop=True)
)
display(df)


df = (
  runs_df[
    (runs_df['model.name'] != "periodical_mlp") &
    (runs_df['dataset.name'].str.startswith("108+43-24h-1h"))
  ]
    .sort_values('Mean_Val_mae')
    .groupby(['model.name'])
    .head(10)
    .groupby(['model.name'])['Mean_Test_mae']
    .mean()
    .reset_index(name='Mean_Test_mae_top10')
    .sort_values('Mean_Test_mae_top10', ascending=True)
    .reset_index(drop=True)
)
display(df)

,model.name,dataset.name,Mean_Test_mae_top10
0,cnn_lstm,108+43-24h-1h-2c,0.078727
1,transformer_base,108+43-24h-1h-2c,0.083602
2,lstm,108+43-24h-1h-2c,0.083680
3,cnn_lstm,108+43-24h-1h-1c,0.093935
4,lstm,108+43-24h-1h-1c,0.096925
5,transformer_base,108+43-24h-1h-1c,0.104206


,model.name,dataset.name,Mean_Test_mae_top10
0,transformer_base,108+43-24h-1h-2c,0.083602
1,transformer_base,108+43-48h-1h-2c,0.083695
2,transformer_base,108+43-72h-1h-2c,0.086317
3,transformer_base,108+43-24h-1h-1c,0.104206
4,transformer_base,108+43-48h-1h-1c,0.104254
5,transformer_base,108+43-72h-1h-1c,0.108848


,model.name,Mean_Test_mae_top10
0,cnn_lstm,0.078727
1,transformer_base,0.083602
2,lstm,0.083680


### Otras Estaciones

In [10]:
#  Estación m-43
#  43 + 108
#  43 + 267
#  43 + 267 + 108


#  Estación m-267
#  267 + 43
#  267 + 109
#  267 + 109 + 43


df = (
  runs_df[
    (runs_df['dataset.name'].str.startswith("43") | runs_df['dataset.name'].str.startswith("267"))
  ]
    .sort_values('Mean_Val_mae')
    .groupby(['model.name', 'dataset.name'])
    .head(10)
    .groupby(['dataset.name', 'model.name'])['Mean_Test_mae']
    .mean()
    .reset_index(name='Mean_Test_mae_top10')
    .sort_values('dataset.name', ascending=True)
    .reset_index(drop=True)
)
df['variant'] = df['dataset.name'].str.extract(r'-(\d+c)$', expand=False).fillna('base')
df['base_name'] = df['dataset.name'].str.replace(r'-(\d+c)$', '', regex=True)

pivot_df = df.pivot_table(index=['base_name', 'model.name'],
              columns='variant',
              values='Mean_Test_mae_top10').reset_index()
pivot_df['Mean_error'] = (pivot_df['1c'] + pivot_df['2c']) / 2

display(pivot_df)

variant,base_name,model.name,1c,2c,Mean_error
0,267+109-24h-1h,cnn_lstm,0.081602,NaN,NaN
1,267+43-24h-1h,cnn_lstm,0.084915,0.090508,0.087712
2,43+108-24h-1h,cnn_lstm,0.085142,0.093368,0.089255
3,43+267+108-24h-1h,cnn_lstm,0.081296,NaN,NaN
4,43+267-24h-1h,cnn_lstm,0.091028,0.093125,0.092077
5,43-24h-24h-1L,cnn_lstm,0.129022,0.127922,0.128472
6,43-24h-24h-2L,cnn_lstm,0.126758,0.127123,0.126940
7,43-24h-24h-3L,cnn_lstm,0.124271,0.123551,0.123911


## Pruebas horizonte 24h


In [11]:
df = (
  runs_df[
    (runs_df['dataset.name'].str.contains(r'h-24h-.*$', regex=True))
  ]
    .sort_values('Mean_Val_mae')
    .groupby(['model.name', 'dataset.name'])
    .head(10)
    .groupby(['dataset.name', 'model.name'])['Mean_Test_mae']
    .mean()
    .reset_index(name='Mean_Test_mae_top10')
    .sort_values('dataset.name', ascending=True)
    .reset_index(drop=True)
)
df['variant'] = df['dataset.name'].str.extract(r'-(\d+c)$', expand=False).fillna('base')
df['base_name'] = df['dataset.name'].str.replace(r'-(\d+c)$', '', regex=True)

pivot_df = df.pivot_table(index=['base_name', 'model.name'],
              columns='variant',
              values='Mean_Test_mae_top10').reset_index()
pivot_df['Mean_error'] = (pivot_df['1c'] + pivot_df['2c']) / 2

display(pivot_df)

variant,base_name,model.name,1c,2c,Mean_error
0,108+43-24h-24h,cnn_lstm,0.130675,0.122878,0.126776
1,108+43-24h-24h-1L,cnn_lstm,0.114363,0.110375,0.112369
2,108+43-24h-24h-2L,cnn_lstm,0.113966,0.108038,0.111002
3,108+43-24h-24h-3L,cnn_lstm,0.111496,0.104304,0.107900
4,108-24h-24h-3L,cnn_lstm,0.110230,0.107251,0.108740
5,139-24h-24h-1L,cnn_lstm,0.095232,0.119764,0.107498
6,139-24h-24h-2L,cnn_lstm,0.094497,0.121361,0.107929
7,139-24h-24h-3L,cnn_lstm,0.090015,0.116881,0.103448
8,139-43-108-24h-24h-1L,cnn_lstm,0.118100,0.132910,0.125505
9,139-43-108-24h-24h-2L,cnn_lstm,0.112987,0.130502,0.121745
